In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")

from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

import math
import os, sys, gc, time, warnings, pickle, psutil, random
import gc
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
from collections import Counter
from tqdm import tqdm



/kaggle/src/script.ipynb
/kaggle/lib/kaggle/gcp.py
/kaggle/input/kakao-arena-melon/train.json
/kaggle/input/kakao-arena-melon/test.json
/kaggle/input/kakao-arena-melon/genre_gn_all.json
/kaggle/input/kakao-arena-melon/song_meta.json
/kaggle/input/kakao-arena-melon/val.json
/kaggle/input/make-pv-file/__results__.html
/kaggle/input/make-pv-file/__notebook__.ipynb
/kaggle/input/make-pv-file/custom.css
/kaggle/input/make-pv-file/train_DF.parquet
/kaggle/input/make-pv-file/__output__.json
/kaggle/input/kakao-train-valid-split/train_y.json
/kaggle/input/kakao-train-valid-split/train_df.json
/kaggle/input/kakao-train-valid-split/val_x.json
/kaggle/input/kakao-train-valid-split/__results__.html
/kaggle/input/kakao-train-valid-split/__notebook__.ipynb
/kaggle/input/kakao-train-valid-split/train_x.json
/kaggle/input/kakao-train-valid-split/custom.css
/kaggle/input/kakao-train-valid-split/__output__.json
/kaggle/input/kakao-train-valid-split/val_y.json
/kaggle/input/kakao-train-valid-split/val_df

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

Using TensorFlow backend.


In [3]:

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


In [4]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath(parent)
    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [5]:
train = pd.read_json('/kaggle/input/kakao-arena-melon/train.json')
train_pv = pd.read_parquet('/kaggle/input/make-pv-file/train_DF.parquet')


In [6]:
# from sklearn import preprocessing
# song_meta = pd.read_json('/kaggle/input/kakao-arena-melon/song_meta.json')

In [7]:


# song_meta['issue_year'] = song_meta['issue_date'].apply(lambda x: str(x)[2:4])

# id_year_dict = pd.Series(song_meta.issue_year.values, index=song_meta.id).to_dict()
# del song_meta
# gc.collect()
# train_pv.reset_index(inplace = True)
# train_pv['year'] = train_pv['song'].apply(lambda x :id_year_dict[x])
# train_pv['year'] = pd.to_numeric(train_pv['year'],errors='coerce')




In [8]:
# x = train_pv['year'].values.reshape(-1,1) #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# x = pd.DataFrame(x_scaled)
# train_pv['year'] = x


# train_pv.set_index('song', inplace = True)

# del song_meta, x
# gc.collect()

In [9]:
# index2id = {i:u for i, u in enumerate((train_pv.index))}
# id2index = {u:i for i, u in enumerate((train_pv.index))}
# train_pv.reset_index(inplace = True, drop = True)

In [10]:
print(train_pv.shape)
display(train_pv.head(2))
display(train_pv.tail(2))

(34242, 18074)


id,6,25,39,50,55,85,106,107,112,127,...,153265,153275,153280,153288,153332,153347,153364,153388,153421,153428
song,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


id,6,25,39,50,55,85,106,107,112,127,...,153265,153275,153280,153288,153332,153347,153364,153388,153421,153428
song,,,,,,,,,,,,,,,,,,,,,
707913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
## make a model
from keras.layers import Input, Dense
from keras.models import Model


In [12]:
# this is the size of our encoded representations
# encoding_dim = 200  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# intput size
input_dim = train_pv.shape[1]

In [13]:
input_dim

18074

## denoising autoencoder

In [14]:

# x_train = train_pv.values

# noise_factor = 0.5
# x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
# # x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 
# x_train_noisy = np.clip(x_train_noisy, 0., 1.)
# # x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [15]:
# a = 10000
# b = 5000
# c = 500
# input_dim = train_pv.shape[1]
# model = Sequential()
# model.add(Dense(a, activation='relu', input_dim=input_dim))
# model.add(Dense(b, activation='relu'))
# model.add(Dense(c, activation='relu'))
# model.add(Dense(b, activation='relu'))
# model.add(Dense(a, activation='relu'))
# model.add(Dense(input_dim, activation='sigmoid'))
# model.compile(optimizer='adam', loss='mse')

In [16]:
# model.fit(x_train_noisy, x_train, 
#           nb_epoch=5,
#           batch_size=256,
#           shuffle=True)

In [17]:
a = 1000
b = 500

input_img = Input(shape=(input_dim,)) ## input layer
encoded = Dense(a, activation='relu')(input_img) ## encoding dim으로 압축
encoded = Dense(b, activation='relu')(encoded) ## encoding dim으로 압축


decoded = Dense(b, activation='sigmoid')(encoded) ## input dim으로 복귀
decoded = Dense(input_dim, activation='sigmoid')(encoded) ## input dim으로 복귀

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# create a placeholder for an encoded (32-dimensional) input01
# encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(b,))

# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

# compile
autoencoder.compile(optimizer='adam', loss='mse') # (optimizer='adadelta', loss='binary_crossentropy')

## train
tmp = train_pv.values
history = autoencoder.fit(tmp, tmp,
                epochs=5,
                batch_size=256,
                shuffle=True)

Epoch 1/5
34242/34242 [==============================] - 167s 5ms/step - loss: 0.0283
Epoch 2/5
34242/34242 [==============================] - 161s 5ms/step - loss: 0.0013
Epoch 3/5
34242/34242 [==============================] - 163s 5ms/step - loss: 0.0013
Epoch 4/5
34242/34242 [==============================] - 171s 5ms/step - loss: 0.0013
Epoch 5/5
34242/34242 [==============================] - 162s 5ms/step - loss: 0.0013


In [18]:
encoded_imgs = encoder.predict(tmp)
# decoded_imgs = decoder.predict(encoded_imgs)

del input_img, autoencoder, encoder, encoded_input, decoder_layer
gc.collect()

292

In [19]:
del tmp
gc.collect()

20

In [20]:
train_data = pd.DataFrame(data=encoded_imgs)

del encoded_imgs
gc.collect()

20

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_array = cosine_similarity(train_data, train_data)
print('cosine_array complete')
# item_based_array= pd.DataFrame(data = cosine_array, index = train_pv.index, columns = train_pv.index)


index2id = {i:u for i, u in enumerate((train_pv.index))}

cf_dict = {}
for i in tqdm(range(len(cosine_array))):
    song_id= list(map(index2id.get, list(np.argsort(-cosine_array[i])[1:201])))
    value= list(np.sort(cosine_array[0])[::-1][1:201])
    cf_dict[index2id[i]] = list(zip(song_id,value))

del cosine_array, train_data
gc.collect()

  0%|          | 15/34242 [00:00<04:01, 141.87it/s]

cosine_array complete


100%|██████████| 34242/34242 [03:30<00:00, 162.85it/s]


0

In [22]:
# print(cosine_array[2][:5])
# print(np.argsort(-cosine_array[2])[:5])
# t= np.argsort(-cosine_array[2])[:5]

In [23]:
# song_meta[song_meta['id'].isin([62,366786,116573])]

## 본격 Predict

In [24]:
def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists[col]:
        c.update(doc)

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

def remove_seen(seen, l):
    seen = set(seen)
    ## 데이터에 l의 원소가 없으면 l의 원소를 채워 넣는다. 
    return [x for x in l if not (x in seen)]

In [25]:
val = pd.read_json('/kaggle/input/kakao-arena-melon/val.json')

In [26]:
def check_target_type(val):
    no_song = []
    no_both = []
    no_tag = []
    yes_index = []

    for i in range(len(val)):
        if (not val.loc[i:i, 'songs'].values[0])&(not not val.loc[i:i, 'tags'].values[0]): ## 노래는 없는데 태그는 있어
            no_song.append(i)

        elif (not not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래는 있는데 태그가 없으
            no_tag.append(i)  

        elif (not val.loc[i:i, 'songs'].values[0]) & (not val.loc[i:i, 'tags'].values[0]): ## 노래도 없고 태그도 없어
            no_both.append(i)          

        else: ## 둘다 있음, 아마도 노래 몇 곡이 누락 됐어 
            yes_index.append(i)
            
    print("노래가 하나도 없음 : ", len(no_song))
    print("노래, 태그 둘 다 존재함 : ", len(yes_index))
    print("태그가 없음 : ",len(no_tag) )
    print("둘다 없음 : ", len(no_both))
    
    return no_song, no_both, no_tag, yes_index
        

In [27]:
no_song, no_both, no_tag, yes_index = check_target_type(val)


노래가 하나도 없음 :  2630
노래, 태그 둘 다 존재함 :  8975
태그가 없음 :  9661
둘다 없음 :  1749


## 3. 둘 다 있음
- 기존에 존재하던 tag에서 가장 빈번하게 나타났던 음악을 넣어주기 

In [28]:
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))


year_list = list(train['year'].unique())
year_list= sorted(year_list)

val3 = val[val.index.isin(yes_index)]


small_years = [17,18,19,20]

val_tmp = val3[val3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)
val_tmp.reset_index(inplace = True, drop = True)

val year shape (6401, 7)


In [29]:
popular_year = []
for i in year_list:
    tmp = train[train['year'] == i]
    _, year_song = most_popular(tmp, "songs", 200)
    _, year_tag = most_popular(tmp, "tags", 100)
    popular_year.append({
        "year" : i,
        "songs" : year_song,
        "tags" : year_tag,})

In [30]:
song_tag_dict = load_json('/kaggle/input/kakao-save-song-tag-dict/song_tag_dict.json')


In [31]:
def f7(seq):
    '''
    list에 있는 중복 데이터는 삭제하고, 순서는 유지하는 함수
    set만 사용하면 순서가 뒤엉키게 됨
    '''
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [32]:
val3_predict = []
for i in range(len(val_tmp)):
    year = val_tmp.loc[i:i, 'year'].values[0]
    
    
    ################ song 채우기 ###############################
    songs = val_tmp.loc[i:i, 'songs'].values[0]
    songs = [x for x in songs if x in cf_dict]

    if len(songs):
        tmp_list = []
        for song in songs:
            tmp_list.extend(cf_dict[song]) 
        tmp_list.sort(key = lambda x: x[1], reverse = True)    
        tmp_list = [i[0] for i in tmp_list if i[1]>0]
        tmp_list = f7(tmp_list)

        cur_song = tmp_list[:200]
    else:
        cur_song = popular_year[year-4]['songs']

    cur_song = remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100]

    #중복 케이스 때문에 부족하면 popular song으로 다시 채워줌
    if len(cur_song) != 100:
        update_song = remove_seen(cur_song, popular_year[year-4]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:200]
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val_tmp.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_year[year-4]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_year[year-4]['tags'][:100]
        
    #############################################################    

    val3_predict.append({
        "id" : list(val_tmp.loc[i:i, 'id'])[0],
        "songs": remove_seen(list(val_tmp.loc[i:i, 'songs'])[0], cur_song)[:100],
        "tags": remove_seen(list(val_tmp.loc[i:i, 'tags'])[0], cur_tag)[:10],
     })    

    


In [33]:

write_json(val3_predict, 'val3_2019_predict.json')

In [34]:
# from IPython.display import FileLink
# FileLink(r'val3_2015_predict.json')